# 동역학 시스템 모델링하기(Modeling Dynamical Systems)
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.

이 notebook은 Drake에서 입력-출력 동적 시스템을 모델링하는 간단한 튜토리얼이다. 다음 내용을 다룬다.
- 나만의 간단한 고유 동적 시스템(dynamical system) 작성
- 동적 시스템 시뮬레이션과 결과 그림 그리기
- 간단한 시스템의 블록 다이어그램 구성

Drake는 drake::systems에 C++ 클래스 세트가 있다. 이 클래스 세트는 간단한 동적 시스템을 블록 다이어그램으로 연결하여 복잡한 동적 시스템을 구축하는 API를 제공합니다. 모델링 접근 방식은 MATLAB의 Simulink에서 영감을 받았지만 그래픽 사용자 인터페이스는 없다. Simulink과 마찬가지로 drake::systems는 기존 시스템 라이브러리를 제공하고 사용자 지정 새로운 동적 시스템을 쉽게 정의할 수 있게 해준다. 이 강의에서 사용되는 `pydrake`의 파이썬 wrapper는 이 시스템 프레임워크 맨 위에서 편리한 바인딩 세트를 제공하여 파이썬이나 C++로 새로운 시스템을 작성하고 상호 교환하여 사용할 수 있게 해준다.

Drake에서 동적 시스템을 모델링하는 것과 Simulink(또는 대부분의 다른 시스템 모델링 언어)의 주요 차이점은 제어 방정식의 기본 구조를 자세히 검토하는 데 중점을 두고 있다는 것이다. 예를 들어, 동적 시스템의 특정 출력이 입력의 작은 부분에만 의존한다면 Drake는 이 희소성을 직접 추론하려고 시도한다. 마찬가지로, 시스템의 동적 특성을 다항식 벡터 필드(polynomial vector field: 또는 선형, 또는 적어도 미분가능, ...)로 설명할 수 있다면 Drake는 이 구조를 추론하고 시스템을 다이어그램으로 결합할 때 이 구조를 유지하려고 한다. 이 구조는 수치 시뮬레이션에만 미미한 이점이 있을 수 있지만 더욱 정교한 분석, 시스템 식별 및 피드백 또는 상태 추정기 설계 알고리즘을 사용할 수 있게 해준다.

나중에 조금 더 고급 모델링 도구로 돌아가겠지만 우선은 몇 가지 매우 간단한 예시부터 살펴보자.y simple examples.

문서에 대한 간단한 메모. C++ 클래스는 [doxygen](http://drake.mit.edu/doxygen_cxx)로 잘 문서화되어 있다. 파이썬 API도 [documented](https://drake.mit.edu/pydrake/index.html)화 되어 있지만 자동 생성되고 항상 잘되어 있지는 않다. 파이썬 개발에서도 C++ doxygen을 보조 문서화 자료로 사용하는 것을 추천한다. `pydrake` API 대부분은 C++ API에 바로 매핑되어 있다.

## A library of dynamical systems

Drake는 기본적인 시스템 빌딩 블록인 적분기(integrators), 멀티플렉서(multiplexers)와 같은 시뮬레이션 물리 엔진 구현 시스템부터 추정, 제어를 위한 고급 알고리즘을 구현한 더 복잡한 시스템 등 다양한 동적 시스템 라이브러리[rich library of dynamical
systems](https://drake.mit.edu/doxygen_cxx/group__systems.html)를 제공한다. 아래에서 보는 바와 같이 이러한 기본 블록은 나중에 더 복잡한 시스템과 결합할 수 있다. 또한 Drake는 (C++나 직접 Python으로) 사용자가 정의하는 시스템을 쉽게 작성할 수 있도록 해준다.

## Writing your own dynamics

이 섹션에서는 `pydrake`에서 사용자가 정의하는 동적 클래스를 작성하는 방법을 설명한다. 모든 사용자 시스템은 `pydrake.systems.framework.LeafSystem` 클래스를 상속받아야 한다.

하지만, 우리가 관심을 갖는 많은 동적 시스템은 상태 공간 형태의 간단한 벡터 필드로 표현된다. 여기서 우리는 종종 x로 상태 벡터를 나타내고, u로 입력 벡터, y로 출력 벡터를 나타낸다. 이러한 형태의 시스템을 더 쉽게 구현할 수 있도록, 간단한 시스템을 쉽게 작성할 수 있게 하는 다른 하위 클래스인 `pydrake.systems.primitives.SymbolicVectorSystem`을 제공합니다.

<!-- The following HTML table format was modified from the doxygen System block output, with the goal of matching the style -->
<table align="center" cellpadding="0" cellspacing="0">
<tr align="center">
<td><table cellspacing="0" cellpadding="0">
<tr>
<td align="right" style="padding:5px 0px 5px 0px">u &rarr; </td></tr>
</table>
</td><td align="center" style="border:solid;padding-left:20px;padding-right:20px" bgcolor="#F0F0F0">System</td><td><table cellspacing="0" cellpadding="0">
<tr>
<td align="left" style="padding:5px 0px 5px 0px">&rarr; y </td></tr>
</table>
</td></tr>
</table>

### SymbolicVectorSystem 사용하기(Using SymbolicVectorSystem)

다음 상태 공간 방정식으로 설명되는 기본적인 연속 시간, 비선형, 입력-출력 동적 시스템(input-output dynamical system)을 고려하시오 :

$\begin{aligned}\dot{x} =& f(t,x,u), \\ y =& g(t,x,u).\end{aligned}$

`pydrake`에서 $f()$ 와 $g()$가 [Drake symbolic engine](https://drake.mit.edu/pydrake/pydrake.symbolic.html#pydrake.symbolic.Expression)에서 지원하는 연산을 사용하여 Python에서 작성할 수 있는 것이라면 무엇이든 이러한 형태의 시스템을 인스턴스화할 수 있다. 다음 예시를 통해 알아보자.

이 시스템을 고려해보자. 

$\begin{aligned}\dot{x} =& -x + x^3,\\ y =& x.\end{aligned}$

이 시스템은 입력이 없고, 하나(연속적인) 상태 변수와 하나의 출력을 가지고 있다. 다음 코드를 사용하여 Drake에서 구현할 수 있다.:

In [ ]:
from pydrake.symbolic import Variable
from pydrake.systems.primitives import SymbolicVectorSystem

# Define a new symbolic Variable
x = Variable("x")

# Define the System.  
continuous_vector_system = SymbolicVectorSystem(state=[x], dynamics=[-x + x**3], output=[x])


이게 끝이다! `continuous_vector_system` 변수는 이제 Drake의 System 클래스의 인스턴스이다. 이 변수는 아래에서 설명할 여러 가지 방법으로 사용할 수 있다. 상태 인자(state argument)는 `symbolic::Variable`의 *벡터*를 기대하고(Python 리스트는 자동 변환됨), 동적 및 출력 인자는 `symbolic::Expression`의 벡터를 기대한다.

Drake에서 기본적인 이산 시간 시스템(discrete-time system)을 구현하는 것은 연속 시간 시스템(continuous-time system)을 구현하는 것과 매우 유사하다. 다음과 같이 주어진 이산 시간 시스템: 

$\begin{gathered}  x[n+1] = f(n,x,u),\\ y[n] = g(n,x,u), \end{gathered}$ 

다음 예제와 같이 구현할 수 있다.

시스템을 고려해보자 

$\begin{gathered}x[n+1] = x^3[n],\\ y[n] = x[n].\end{gathered}$

이 시스템은 입력이 없고, 하나(이산) 상태 변수와 하나의 출력을 가지고 있다. 다음 코드를 사용하여 Drake에서 구현할 수 있다.:

In [ ]:
from pydrake.symbolic import Variable
from pydrake.systems.primitives import SymbolicVectorSystem

# Define a new symbolic Variable
x = Variable("x")

# Define the System.  Note the additional argument specifying the time period.
discrete_vector_system = SymbolicVectorSystem(state=[x], dynamics=[x**3], output=[x], time_period=1.0)

### LeafSystem 상속받기(Deriving from LeafSystem)

SymbolicVectorSystems를 사용하는 것은 시작하기 좋은 방법이지만, 실제로 Drake는 다중 입력 및 출력, 혼합 이산 및 연속 동적, 경계 조건과 재설정이 있는 하이브리드 동적, 제약 조건이 있는 시스템, 심지어 확률 시스템과 같이 다양한 시스템을 작성할 수 있다. 기본 시스템 프레임워크보다 더 다양한 일을 하려면 단순화된 `SymbolicVectorSystem` 인터페이스 대신에 직접 `pydrake.systems.framework.LeafSystem`를 상속받을 수 있다.

In [ ]:
from pydrake.systems.framework import LeafSystem

# Define the system.
class SimpleContinuousTimeSystem(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)

        state_index = self.DeclareContinuousState(1)  # One state variable.
        self.DeclareStateOutputPort("y", state_index)  # One output: y=x.

    # xdot(t) = -x(t) + x^3(t)
    def DoCalcTimeDerivatives(self, context, derivatives):
        x = context.get_continuous_state_vector().GetAtIndex(0)
        xdot = -x + x**3
        derivatives.get_mutable_vector().SetAtIndex(0, xdot)

# Instantiate the System
continuous_system = SimpleContinuousTimeSystem()


# Define the system.
class SimpleDiscreteTimeSystem(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)

        state_index = self.DeclareDiscreteState(1)  # One state variable.
        self.DeclareStateOutputPort("y", state_index)  # One output: y=x.
        self.DeclarePeriodicDiscreteUpdateEvent(
            period_sec=1.0,  # One second time step.
            offset_sec=0.0,  # The first event is at time zero.
            update=self.Update) # Call the Update method defined below.

    # x[n+1] = x^3[n]
    def Update(self, context, discrete_state):
        x = context.get_discrete_state_vector().GetAtIndex(0)
        x_next = x**3
        discrete_state.get_mutable_vector().SetAtIndex(0, x_next)

# Instantiate the System
discrete_system = SimpleDiscreteTimeSystem()

이 코드는 위에서 구현한 것과 동일한 시스템을 구현한다. `SymbolicVectorSystem` 버전과 달리, 여기에 있는 method에 유효한 Python/numpy 코드를 입력할 수 있으며, `pydrake.symbolic`에서 지원할 필요가 없다. 또한, 이제 `LeafSystem`에서 훨씬 더 많은 기능을 오버로드하여 훨씬 더 복잡한 시스템(다중 입력 및 출력 포트, 혼합 이산 및 연속 상태, 더 구조화된 상태, witness 함수 및 reset 맵이 있는 하이브리드 시스템 등)을 생성할 수 있다. 그러나 이러한 방식으로 `LeafSystem`을 선언한다고 해서 Drake의 자동 미분(autodiff) 및 기호 도구(symbolic tools)에 대한 즉각적인 지원이 제공되지는 않는다. 이를 위해서는 [템플릿을 지원하기 위해 몇 줄을](https://drake.mit.edu/pydrake/pydrake.systems.scalar_conversion.html?highlight=templatesystem#pydrake.systems.scalar_conversion.TemplateSystem) 더 추가해야 한다.

또한 [`pydrake.systems.primitives.LinearSystem`](https://drake.mit.edu/pydrake/pydrake.systems.primitives.html?highlight=linearsystem#pydrake.systems.primitives.LinearSystem) 클래스 또는 [`pydrake.systems.primitives.Linearize()`](https://drake.mit.edu/pydrake/pydrake.systems.primitives.html?highlight=linearize#pydrake.systems.primitives.Linearize) 메서드와 같이 `LeafSystem`에서 파생되거나 `LeafSystem`을 생성하는 [여러 가지 도우미 클래스 및 메서드](https://drake.mit.edu/doxygen_cxx/group__systems.html)를 제공한다. 로봇의 동역학 및 액추에이터/센서 시뮬레이션과 같이 다양한 경우에 필요한 대부분의 클래스가 이미 구현되어 있다.

`LeafSystem`의 고급 기능을 사용하는 더 많은 예를 보려면 다음으로 [Authoring leaf systems tutorial](./authoring_leaf_systems.ipynb)을 읽어보자.

## Simulation

일단 관심 대상의 동역학을 서술하는 `System` 객체를 획득하게 된 후에, 우리가 할 수 있는 가장 기본적으로 할 수 있는 것은 이를 시뮬레이션하는 것이다. 이는 `pydrake.framework.analysis.Simulator` 클래스를 통해 수행된다. 이 클래스는 가변 단계 적분(variable-step integration), stiff solver 및 이벤트 감지를 지원하는 다양한 수치 적분 루틴(numerical integration routines)에 대한 액세스를 제공한다.

시뮬레이션 실행 후 시뮬레이션으로부터의 데이터를 보려면 `pydrake.framework.primitives.VectorLogSink` 시스템을 다이어그램에 추가해야 한다.

다음 코드를 사용하여 위에서 정의한 연속 시간 시스템을 시뮬레이션하고 결과를 그려보자.:

In [ ]:
import matplotlib.pyplot as plt
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.primitives import LogVectorOutput

# Create a simple block diagram containing our system.
builder = DiagramBuilder()
system = builder.AddSystem(SimpleContinuousTimeSystem())
logger = LogVectorOutput(system.get_output_port(0), builder)
diagram = builder.Build()

# Set the initial conditions, x(0).
context = diagram.CreateDefaultContext()
context.SetContinuousState([0.9])

# Create the simulator, and simulate for 10 seconds.
simulator = Simulator(diagram, context)
simulator.AdvanceTo(10)

# Plot the results.
log = logger.FindLog(context)
plt.figure()
plt.plot(log.sample_times(), log.data().transpose())
plt.xlabel('t')
plt.ylabel('y(t)');

In [ ]:
# Create a simple block diagram containing our system.
builder = DiagramBuilder()
system = builder.AddSystem(SimpleDiscreteTimeSystem())
logger = LogVectorOutput(system.get_output_port(0), builder)
diagram = builder.Build()

# Create the simulator.
simulator = Simulator(diagram)

# Set the initial conditions, x(0).
state = simulator.get_mutable_context().get_mutable_discrete_state_vector()
state.SetFromVector([0.9])

# Simulate for 10 seconds.
simulator.AdvanceTo(10)

# Plot the results.
log = logger.FindLog(simulator.get_context())
plt.figure()
plt.stem(log.sample_times(), log.data().transpose())
plt.xlabel('n')
plt.ylabel('y[n]');

SymbolicVectorSystem` 버전으로 대체해보자. 마찬가지로 작동한다.

많은 시스템에서 시뮬레이션은 실시간보다 훨씬 빠르게 실행된다. 시뮬레이터가 가능한 경우 실시간 시계의 몇 배 느리게 실행하도록 지시하려면 `Simulator`의 `set_target_realtime_rate()` 메서드를 사용한다. 이는 예를 들어 시뮬레이션하는 로봇을 애니메이션화하고 물리적 직관을 얻으려고 하거나 시뮬레이션을 다중 프로세스 실시간 제어 시스템(multi-process real-time control system)의 일부로 사용하려는 경우 유용하다.

## The System "Context"

주의 깊게 살펴보았다면 위 코드 조각에서 "context"라는 단어가 몇 번 등장한 것을 눈치채을 것이다. [`Context`](http://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_context.html) 는 Drake 시스템 프레임워크의 핵심 개념이다. `Context`는 `System`이 핵심 방법을 구현하는 데 필요한 모든 (잠재적으로) 동적 정보를 캡처한다. 여기에는 시간, 상태, 입력 및 시스템 매개변수가 포함된다. `System`의 `Context`는 시뮬레이션(또는 제어 설계 등)에 필요한 모든 것이며, `Context`가 주어지면 `System`에서 호출되는 모든 메서드는 완전히 결정적/반복 가능해야만 한다.

`System`은 `Context`의 인스턴스를 만드는 방법을 알고 있다(참조: [CreateDefaultContext](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_system.html#ad047317ab91889c6743d5e47a64c7f08)). 위의 시뮬레이션 예에서 `Simulator`가 우리를 위해 `Context`를 생성했다. 시뮬레이션을 실행하기 전에 시스템의 초기 조건(상태)을 설정하기 위해 `Simulator`에서 `Context`를 가져왔다.

모든 입력 포트가 연결되지 않으면 `Context`가 완전히 정의되지 않는다(입력 포트 전부가 연결되지 않으면 시뮬레이션 및 기타 메서드 호출이 실패한다). 다른 시스템의 출력에 직접 연결되지 않은 입력 포트에 대해서는 포트의 [`FixValue`](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_input_port.html#ab285168d3a19d8ed367e11053aec79c3) 메서드를 사용하는 것을 고려하자.

## 시스템들 조합하기(Combinations of Systems: Diagram and DiagramBuilder)

Drake의 진정한 모델링 파워는 많은 작은 시스템을 결합하여 더 복잡한 시스템으로 만드는 데 있다. 이 개념은 매우 간단하다. `DiagramBuilder` 클래스를 사용하여 `AddSystem()`을 호출하고 입력 포트를 출력 포트에 `Connect()`하거나 다이어그램의 입력/출력으로 노출한다. 그런 다음, `Build()`를 호출하여 프레임워크 내의 또 다른 System인 새로운 `Diagram` 인스턴스를 생성하고 전체 도구 세트를 사용하여 시뮬레이션 또는 분석할 수 있다.

아래 예시에서는 세 개의 서브시스템(plant, controller, logger)을 연결하고, 생성되는 `Diagram`에 대한 입력으로 컨트롤러의 입력을 노출한다. :

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pydot
from IPython.display import SVG, display
from pydrake.examples import PendulumPlant
from pydrake.systems.analysis import Simulator
from pydrake.systems.controllers import PidController
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.primitives import LogVectorOutput

builder = DiagramBuilder()

# First add the pendulum.
pendulum = builder.AddNamedSystem("pendulum", PendulumPlant())

# Add a PID controller.
controller = builder.AddNamedSystem("controller",
                                    PidController(kp=[10.], ki=[1.], kd=[1.]))

# Now "wire up" the controller to the plant.
builder.Connect(pendulum.get_state_output_port(),
                controller.get_input_port_estimated_state())
builder.Connect(controller.get_output_port_control(), pendulum.get_input_port())

# Make the desired_state input of the controller an input to the diagram.
builder.ExportInput(controller.get_input_port_desired_state())
# Make the pendulum state an output from the diagram.
builder.ExportOutput(pendulum.get_state_output_port())

# Log the state of the pendulum.
logger = LogVectorOutput(pendulum.get_state_output_port(), builder)
logger.set_name("logger")

diagram = builder.Build()
diagram.set_name("diagram")

# Visualize the diagram.
display(SVG(pydot.graph_from_dot_data(
    diagram.GetGraphvizString(max_depth=2))[0].create_svg()))


다른 모든 `System`과 마찬가지로 `Diagram`도 `Context`을 가진다. `Diagram.GetSubSystemContext` 또는 `Diagram.GetMutableSubsystemContext()`를 사용하여 하위 시스템의 컨텍스트를 항상 추출할 수 있다. (아직 명확하지 않은 경우 "mutable"은 쓰기 권한이 있음을 의미한다. 그렇지 않으면 데이터는 `const`로 간주된다. 시스템 프레임워크에서 알고리즘/사용자가 기본 시스템 추상화를 위반하지 않도록 보호하기 위해 어느 정도 노력한다.) 불행히도 C++에서는 컴파일러가 `const` 속성을 강제하지만 Python은 이를 무시한다. Python에서 "mutable"을 사용하는 것은 우리의 의도를 명확히 하고 C++ 코드와의 일관성을 유지하기 위해서이다.

마지막으로 PID 제어 시스템을 시뮬레이션하고 출력을 시각화할 수 있다.

In [ ]:
# Set up a simulator to run this diagram.
simulator = Simulator(diagram)
context = simulator.get_mutable_context()

# We'll try to regulate the pendulum to a particular angle.
desired_angle = np.pi/2.

# First we extract the subsystem context for the pendulum.
pendulum_context = diagram.GetMutableSubsystemContext(pendulum, context)
# Then we can set the pendulum state, which is (theta, thetadot).
pendulum_context.get_mutable_continuous_state_vector().SetFromVector(
    [desired_angle + 0.1, 0.2])

# The diagram has a single input port (port index 0), which is the desired_state.
diagram.get_input_port(0).FixValue(context, [desired_angle, 0.])

# Clear the logger only because we've written this notebook with the opportunity to
# simulate multiple times (in this cell) using the same logger object.  This is
# often not needed.
logger.FindMutableLog(context).Clear()

# Simulate for 10 seconds.
simulator.AdvanceTo(20);

# Plot the results.
log = logger.FindLog(simulator.get_context())
t = log.sample_times()
plt.figure()
# Plot theta.
plt.plot(t, log.data()[0,:],'.-')
# Draw a line for the desired angle.
plt.plot([t[0], t[-1]], [desired_angle, desired_angle], 'g' )
plt.xlabel('time (seconds)')
plt.ylabel('theta (rad)')
plt.title('PID Control of the Pendulum');

## 고급 tutorials
- [Authoring Leaf Systems](./authoring_leaf_systems.ipynb)
- [Working with Diagrams](./working_with_diagrams.ipynb)
